<a href="https://colab.research.google.com/github/utkarsh599/Recomendation-sysy/blob/Abhijeet/learningSpark6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
!pip install py4j

  Using cached https://files.pythonhosted.org/packages/04/de/2d314a921ef4c20b283e1de94e0780273678caac901564df06b948e4ba9b/py4j-0.10.8.1-py2.py3-none-any.whl


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()

In [0]:
import pyspark

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
comments1 =  spark.read.csv("/content/gdrive/My Drive/Colab Notebooks/GBcomments.csv", inferSchema=True, header=True)

In [0]:
comments2= spark.read.csv("/content/gdrive/My Drive/Colab Notebooks/UScomments.csv", inferSchema=True, header=True)

In [11]:
comments1.show()

+-----------+--------------------+-----+-------+
|   video_id|        comment_text|likes|replies|
+-----------+--------------------+-----+-------+
|jt2OHQh0HoQ|It's more accurat...|    0|      0|
|jt2OHQh0HoQ|To be there with ...|    1|      0|
|jt2OHQh0HoQ|Thank gosh, a pla...|    0|      0|
|jt2OHQh0HoQ|What happened to ...|    0|      0|
|jt2OHQh0HoQ|Power is the dise...|    0|      0|
|jt2OHQh0HoQ|Keep calm and buy...|    0|      0|
|jt2OHQh0HoQ|i am a big fan of...|    0|      0|
|jt2OHQh0HoQ|You will never fi...|    0|      0|
|jt2OHQh0HoQ|*APPLE JUST COMME...|    0|      0|
|jt2OHQh0HoQ|I'm only here to ...|    0|      0|
|jt2OHQh0HoQ|Mom said: Just on...|    0|      0|
|jt2OHQh0HoQ|what is the song?...|    0|      0|
|jt2OHQh0HoQ|I started crying ...|    0|      0|
|jt2OHQh0HoQ|3 years late on w...|    0|      0|
|jt2OHQh0HoQ|more one before t...|    0|      0|
|jt2OHQh0HoQ|subscribe me if y...|    0|      0|
|jt2OHQh0HoQ|They really just ...|    0|      0|
|jt2OHQh0HoQ|so if s

In [12]:
type(comments1)

pyspark.sql.dataframe.DataFrame

In [0]:
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

comments=unionAll(comments1,comments2)

In [14]:
comments.count()

1410466

In [40]:
comments_rdd = comments.select('comment_text').rdd.flatMap(lambda x: x)
comments_rdd=comments_rdd.filter(lambda x: x is not None)
comments_rdd.collect()

["It's more accurate to call it the M+ (1000) because the price is closer than calling it the X (10).",
 'To be there with a samsung phone\\n😂😂😂',
 'Thank gosh, a place I can watch it without having to be at HD... my speed doesn’t support HD',
 'What happened to the home button on the iPhone X? *****Cough****copying Samsung******cough',
 'Power is the disease.\xa0 Care is the cure.\xa0 Keep caring for yourself and others as best as you can.\xa0 This is life.',
 'Keep calm and buy iphone 8 Keep calm and buy iphone 8 plus Keep calm and buy iphone X. What is your favourite?',
 'i am a big fan of youtube and u !!!!!!!!!!!!!',
 'You will never find Losers who line up and pay good money for this crap line up to do charity or community work.',
 "*APPLE JUST COMMENTED ON MY LAST VIDEO* I'm crying right now 😭😭😭😢😢😢",
 "I'm only here to see Emma, I love her so much! i'm so proud of her! :)",
 'Mom said: Just one more video before bed',
 'what is the song?? 39:20',
 "I started crying at 49:01 when

In [0]:
header = comments_rdd.first()

In [0]:
data_rmv_col = comments_rdd.filter(lambda row: row != header)

In [0]:
def lower(x):
  return x.lower()

In [0]:
lowerCase_sentRDD = data_rmv_col.map(lower)

In [45]:
lowerCase_sentRDD.collect()

['to be there with a samsung phone\\n😂😂😂',
 'thank gosh, a place i can watch it without having to be at hd... my speed doesn’t support hd',
 'what happened to the home button on the iphone x? *****cough****copying samsung******cough',
 'power is the disease.\xa0 care is the cure.\xa0 keep caring for yourself and others as best as you can.\xa0 this is life.',
 'keep calm and buy iphone 8 keep calm and buy iphone 8 plus keep calm and buy iphone x. what is your favourite?',
 'i am a big fan of youtube and u !!!!!!!!!!!!!',
 'you will never find losers who line up and pay good money for this crap line up to do charity or community work.',
 "*apple just commented on my last video* i'm crying right now 😭😭😭😢😢😢",
 "i'm only here to see emma, i love her so much! i'm so proud of her! :)",
 'mom said: just one more video before bed',
 'what is the song?? 39:20',
 "i started crying at 49:01 when emma's ep showed up ❤️❤️ i'm so proud of her",
 '3 years late on wireless charging, 3 years late on ole

In [0]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [0]:
def sent_TokenizeFunct(x):
    doc=nlp(x)
    tokens=[]
    for token in doc:
      if((token.is_stop==False and token.is_punct==False) and token.is_alpha==True):
        tokens.append(token.lemma_)
    return ' '.join(tokens)
    
sentenceTokenizeRDD = lowerCase_sentRDD.map(sent_TokenizeFunct)

In [0]:
sentenceTokenizeRDD.collect()